In [5]:
import pandas as pd
from google.cloud import bigquery
import pandas as pd 
import chime   
import seaborn as sns
import pymc as pm
import numpy as np
client = bigquery.Client("gcp-wow-rwds-ai-checkout-dev")

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/opt/homebrew/lib/python3.11/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
def store_matching(df):
    # Helper function for greedy matching
    def greedy_matching(store_pairs):
        matched_pairs = []
        used_test_stores = set()
        used_control_stores = set()

        for _, row in store_pairs.iterrows():
            if row['test_store'] not in used_test_stores and row['control_store'] not in used_control_stores:
                matched_pairs.append(row)
                used_test_stores.add(row['test_store'])
                used_control_stores.add(row['control_store'])

        return pd.DataFrame(matched_pairs)

    # Helper function for global matching
    def global_matching(store_pairs):
        store_pairs_sorted = store_pairs.sort_values('abs_perc_diff')
        return greedy_matching(store_pairs_sorted)

    # Calculate total difference
    def calculate_total_difference(matched_pairs):
        return matched_pairs['abs_perc_diff'].sum()

    # Perform greedy matching
    greedy_result = greedy_matching(df)
    greedy_total_diff = calculate_total_difference(greedy_result)

    # Perform global matching
    global_result = global_matching(df)
    global_total_diff = calculate_total_difference(global_result)

    # Print results
    print(f"Greedy Matching Total Difference: {greedy_total_diff}")
    print(f"Global Matching Total Difference: {global_total_diff}")

    # Return the global matching result as a DataFrame
    return global_result



In [9]:
#matches_input = pd.read_csv("./inputs/WOW20001833_1_Store_Pairs.csv")
#matches_input
check = client.query(
    """
    CREATE OR REPLACE TABLE gcp-wow-cart-data-dev-d4d7.davide.example_stores_to_be_matched AS 
        with step_one AS (
        SELECT baseline.*, baseline.sales_amount / baseline.weeks_count AS weekly_avg_sales_amount, campaign.test_or_control --campaign_id, Site, stddev_sales_amount, sales_amount / weeks_count AS weekly_avg_sales_amount
        FROM gcp-wow-cart-data-dev-d4d7.davide.baseline_statistics_with_campaign_3 baseline
        LEFT JOIN (SELECT DISTINCT campaign_id, Site, test_or_control FROM gcp-wow-cart-data-dev-d4d7.davide.regularization_test_campaign_period_transactions) campaign
            ON baseline.campaign_id = campaign.campaign_id 
            AND baseline.Site = campaign.Site
        WHERE baseline.campaign_id = 'WOW20001833_1'
        AND weeks_count = 13
        ) 
        SELECT 
            test.campaign_id AS study_id, 
            test.Site AS test_store, 
            control.Site AS control_store, 
            ABS(test.weekly_avg_sales_amount / control.weekly_avg_sales_amount - 1) + ABS(test.stddev_sales_amount / control.stddev_sales_amount - 1) AS abs_perc_diff
        FROM step_one test
        LEFT JOIN step_one control 
            ON test.Site <> control.Site 
        WHERE test.test_or_control = "Test" 
        AND control.test_or_control = "Control" 
    ;

    SELECT * FROM gcp-wow-cart-data-dev-d4d7.davide.example_stores_to_be_matched ORDER BY test_store;
    """
).result()
matches_input = check.to_dataframe()
matches_input

,study_id,test_store,control_store,abs_perc_diff
0,WOW20001833_1,1003,2604,4.381470
1,WOW20001833_1,1003,5648,0.742072
2,WOW20001833_1,1003,3807,3.588602
3,WOW20001833_1,1003,2647,5.478538
4,WOW20001833_1,1003,2552,2.243831
...,...,...,...,...
76009,WOW20001833_1,7210,2883,0.248937
76010,WOW20001833_1,7210,3267,0.217452
76011,WOW20001833_1,7210,3395,0.141239
76012,WOW20001833_1,7210,5691,0.434665


In [10]:
result_df = store_matching(matches_input)
result_df

Greedy Matching Total Difference: 249.7167452695983
Global Matching Total Difference: 1.9583887764980723


,study_id,test_store,control_store,abs_perc_diff
51074,WOW20001833_1,3390,3058,0.002136
68845,WOW20001833_1,5547,2697,0.002793
20512,WOW20001833_1,2566,4604,0.004133
1098,WOW20001833_1,1030,1336,0.004428
26560,WOW20001833_1,2621,1434,0.004764
...,...,...,...,...
61293,WOW20001833_1,4340,1145,0.072651
63538,WOW20001833_1,4364,5603,0.085762
28174,WOW20001833_1,2642,7207,0.087122
16023,WOW20001833_1,2509,1369,0.091395
